# Advanced Features

## Caching

### Density Functional Theory

In [1]:
import pyiron_core
from pyiron_nodes.atomistic.structure.build import Bulk
from pyiron_nodes.atomistic.engine.quantumespresso import calculate_qe

In [2]:
wf = pyiron_core.Workflow("qe")
wf.structure = Bulk(name="Al", a=4.04, cubic=False)
wf.quantum_espresso = calculate_qe(
    working_directory="test", 
    pseudopotentials="Al.pbe-n-kjpaw_psl.1.0.0.UPF", 
    structure=wf.structure, 
    encut=20, 
    kpts=(3, 3, 3), 
    store=True,
)
pyiron_core.PyironFlow(wf_list=[wf], nodes_path="pyiron_nodes").gui

added node path:  /home/jovyan


![quantumespresso](img/quantum_espresso.png)

### Technical Implementation

In [3]:
@pyiron_core.as_function_node
def long_running_function(i):
    import time 

    time.sleep(10)
    return i

In [4]:
wf = pyiron_core.Workflow("long")
wf.sleep = long_running_function(i=1)
wf.sleep.pull()

1

In [5]:
wf = pyiron_core.Workflow("long")
wf.sleep = long_running_function(i=1)
wf.sleep.pull()

1

In [6]:
@pyiron_core.as_function_node
def long_running_function(i, store=True):
    import time 

    time.sleep(10)
    return i

In [7]:
wf = pyiron_core.Workflow("long")
wf.sleep = long_running_function(i=1)
wf.sleep.pull()

Restoring node outputs  62511dc60d705bf736d5a2cd1529ea3acbfaffcb6e9c2e8c65b1ce715062f2b3 sleep True


1

In [8]:
wf = pyiron_core.Workflow("long")
wf.sleep = long_running_function(i=1)
wf.sleep.pull()

Restoring node outputs  62511dc60d705bf736d5a2cd1529ea3acbfaffcb6e9c2e8c65b1ce715062f2b3 sleep True


1

## Convergence

In [9]:
from pyiron_nodes.atomistic.engine.quantumespresso import converge_energy_cutoff

In [10]:
wf = pyiron_core.Workflow("convergence")
wf.structure = Bulk(name="Al", a=4.04, cubic=False)
wf.quantum_espresso = calculate_qe(
    working_directory="test", 
    pseudopotentials="Al.pbe-n-kjpaw_psl.1.0.0.UPF", 
    structure=wf.structure, 
    encut=20, 
    kpts=(3, 3, 3), 
    store=True,
)
wf.convergence = converge_energy_cutoff(dft_function=wf.quantum_espresso, limit=0.0001, max_steps=10)
pyiron_core.PyironFlow(wf_list=[wf], nodes_path="pyiron_nodes").gui

implement connect to self
connected to node (self)


![convergence](img/convergence.png)

### Technical Implementation 

In [11]:
@pyiron_core.as_function_node
def recursive(x: int, stop_at: int = 10) -> tuple[int, bool]:
    """Toy example for a recursive function."""
    x_new = x + 1

    break_condition = False
    if x_new > stop_at:
        break_condition = True
    return x_new, break_condition

In [12]:
@pyiron_core.as_function_node
def loop_until(recursive_function: pyiron_core.Node, max_steps: int = 10):
    x = recursive_function.inputs.x.value
    for i in range(max_steps):
        x, break_condition = recursive_function(x)
        print("loop: ", i, x, break_condition)

        if break_condition:
            break

    return x

In [13]:
wf = pyiron_core.Workflow("whileloop")
wf.recursive_node = recursive(x=0, stop_at=10)
wf.loop = loop_until(recursive_function=wf.recursive_node, max_steps=20)

In [14]:
wf.loop.pull()

copy node:  recursive_node a5a6903b37da4b61d7cbc8c13c2a0e3fb37340ca2e1aeba088c74fef39ed2f25
loop:  0 1 False
loop:  1 2 False
loop:  2 3 False
loop:  3 4 False
loop:  4 5 False
loop:  5 6 False
loop:  6 7 False
loop:  7 8 False
loop:  8 9 False
loop:  9 10 False
loop:  10 11 True


11

## Energy Volume Curve

In [15]:
import pandas as pd
from pyiron_nodes.basic.executor import SingleNodeExecutor
from pyiron_nodes.atomistic.structure.group import generate_structures
from pyiron_nodes.atomistic.calculator.evcurve import PlotEVcurve
from pyiron_nodes.basic.math import Linspace
from pyiron_nodes.basic.loop import IterToDataFrame

In [16]:
wf = pyiron_core.Workflow("evcurve")
wf.strains = Linspace(x_min=0.9, x_max=1.1, num_points=5)
wf.structure = Bulk(name="Al", a=4.04, cubic=False)
wf.structure_lst = generate_structures(structure=wf.structure, strain_lst=wf.strains)
wf.quantum_espresso = calculate_qe(
    working_directory="test", 
    pseudopotentials="Al.pbe-n-kjpaw_psl.1.0.0.UPF", 
    structure=wf.structure, 
    encut=20, 
    kpts=(3, 3, 3), 
    store=True,
)
wf.exe = SingleNodeExecutor()
wf.df = IterToDataFrame(wf.quantum_espresso, input_label="structure", values=wf.structure_lst, executor=wf.exe)
wf.plot = PlotEVcurve(df=wf.df)
pyiron_core.PyironFlow(wf_list=[wf], nodes_path="pyiron_nodes").gui

implement connect to self
connected to node (self)


![evcurve](img/evcurve.png)

### Technical Implementation

In [17]:
@pyiron_core.as_function_node("range")
def Range(start: int, stop: int, step: int):
    return list(range(start, stop, step))

In [18]:
@pyiron_core.as_function_node
def echo(i=0):
    product = i*i
    return product

In [19]:
wf = pyiron_core.Workflow("parallel")
wf.range_of_int = Range(1, 10, 1)
wf.exe = SingleNodeExecutor()
wf.echo = echo(1)
wf.df = IterToDataFrame(wf.echo, input_label="i", values=wf.range_of_int)
wf.df.pull()

copy node:  echo 9537064e20b039ec9361e245f7516c29260c6c433ce075e27e76f2e563cbbc45


,i,product
0,1,1
1,2,4
2,3,9
3,4,16
4,5,25
5,6,36
6,7,49
7,8,64
8,9,81
